In [1]:
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool
from qwen_agent.utils.output_beautify import typewriter_print
import re
from sql import run_sql_workflow
import time
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.p

In [2]:
llm_cfg = {
    'model': 'Qwen3:8b',
    'model_server': 'http://localhost:11434/v1',
    'generate_cfg': {
        'temperature': 0.0,
    },
}
def remove_think_blocks(text: str) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

In [3]:
@register_tool('get_cases_schema')
class GetCasesSchema(BaseTool):
    description = (
    "Returns the structure and usage instructions for the `cases` table, "
    "which captures core metadata about each procurement or fulfillment case, including dates, supplier info, and performance metrics. "
    "Ideal for analyzing timelines, supplier behavior, case volume, and delivery performance."
)
    parameters = []

    def call(self, params: str, **kwargs) -> str:
        # No parameters needed, but still need to parse for consistency
        _ = json5.loads(params) if params else {}

        return json5.dumps({
            'schema': """
=== CASES TABLE ===

Structure:
    id                     VARCHAR       -- Unique identifier for each case
    order_date             TIMESTAMP_NS  -- Date the order was placed
    employee_id            VARCHAR       -- ID of the employee handling the case
    branch                 VARCHAR       -- Branch responsible for the case
    supplier               VARCHAR       -- Supplier associated with the case
    avg_time               DOUBLE        -- Average time to complete the case
    estimated_delivery     TIMESTAMP_NS  -- Estimated delivery date
    delivery               TIMESTAMP_NS  -- Actual delivery date
    on_time                BOOLEAN       -- Whether the delivery late, False means late delivery
    in_full                BOOLEAN       -- Whether the delivery was complete
    number_of_items        INTEGER       -- Total items in the case
    ft_items               INTEGER       -- Fast-track items
    total_price            DOUBLE        -- Total price of the case
    total_activities       INTEGER       -- Total activities in the process
    rework_activities      INTEGER       -- Count of rework activities
    automatic_activities   INTEGER       -- Count of automated activities

Instructions:
- Use standard SQL (SELECT, WHERE, GROUP BY, etc.).
- Use `order_date` for filtering by time.
- Use aggregations (COUNT, SUM, AVG) for metrics.
"""
        }, ensure_ascii=False)

@register_tool('get_activities_schema')
class GetActivitiesSchema(BaseTool):
    description = (
    "Returns the structure and usage instructions for the `activities` table, "
    "which logs each step performed in a case, including who performed it, when, and whether it was automated or reworked. "
    "Useful for analyzing process flow, bottlenecks, user behavior, and automation levels."
)
    parameters = []

    def call(self, params: str, **kwargs) -> str:
        # No parameters needed, but still need to parse for consistency
        _ = json5.loads(params) if params else {}

        return json5.dumps({
            'schema': """
=== ACTIVITIES TABLE ===


Structure:
    id                       INTEGER     -- Unique ID for each activity
    timestamp                TIMESTAMP   -- Time the activity occurred
    name                     VARCHAR     -- Name/type of activity
    tpt                      DOUBLE      -- Time per task
    user                     VARCHAR     -- User who performed the activity
    user_type                VARCHAR     -- Type of user (e.g., Human, Bot)
    automatic                BOOLEAN     -- Whether the activity was automated
    rework                   BOOLEAN     -- Whether the activity was a rework
    case_index               INTEGER     -- Index of the activity in the case
    case_id                  VARCHAR     -- ID of the related case

Case metadata (prefixed with `case_`):
    case_order_date          TIMESTAMP   -- Order date of the case
    case_employee_id         VARCHAR     -- Employee responsible
    case_branch              VARCHAR     -- Responsible branch
    case_supplier            VARCHAR     -- Supplier involved
    case_avg_time            DOUBLE      -- Average processing time
    case_estimated_delivery  TIMESTAMP   -- Expected delivery date
    case_delivery            TIMESTAMP   -- Actual delivery date
    case_on_time             BOOLEAN     -- Whether the case was on time
    case_in_full             BOOLEAN     -- Whether the delivery was complete
    case_number_of_items     INTEGER     -- Number of items in the case
    case_ft_items            INTEGER     -- Fast-track items
    case_total_price         DOUBLE      -- Total case price

Instructions:
- Use standard SQL syntax (WHERE, GROUP BY, etc.).
- Use `automatic` and `rework` to analyze activities' automation and rework status.
- Use `timestamp`, `name`, or `user_type` for filtering or grouping activities.
- You may aggregate case-related columns, but avoid referencing other tables.
"""
        }, ensure_ascii=False)

@register_tool('get_variants_schema')
class GetVariantsSchema(BaseTool):
    description = (
        "Returns the structure and usage instructions for the `variants` table, "
        "which describes unique sequences of activities (process variants) followed by cases. "
        "Useful for analyzing common paths, deviations, and average processing times in process mining."
    )
    parameters = []

    def call(self, params: str, **kwargs) -> str:
        # No parameters needed, but still need to parse for consistency
        _ = json5.loads(params) if params else {}

        return json5.dumps({
            'schema': """
=== VARIANTS TABLE ===


Structure:
    id              BIGINT       -- Unique ID for each variant
    activities      VARCHAR[]    -- Ordered list of activity names in this variant
    cases           VARCHAR[]    -- Array of case IDs following this variant
    number_cases    BIGINT       -- Number of cases following this variant
    percentage      DOUBLE       -- Share of total cases for this variant
    avg_time        DOUBLE       -- Average processing time for this variant

Instructions:
- Each row represents a unique process path ("variant") followed by one or more cases.
- Use `number_cases`, `percentage`, or `avg_time` to rank, filter, or compare variants.
- Use array functions (e.g., `ANY`, `UNNEST`, `array_length`) to inspect activities or case IDs.
- Standard SQL syntax is allowed (WHERE, ORDER BY, LIMIT, etc.).
- A deviation is consider as all variants except the most common path (the one with the most number of cases).
- To find deviation points you need to compare the activities from the most common path and the variant you're taking as a deviation.
- When the user wants to know about deviations just choose the most common deviations (variants that most cases follow but are different from the most common one).
- To find the impact of a deviation simply take the difference between its average time and the average time for the most common variant.
- Do not reference other tables.
"""
        }, ensure_ascii=False)

@register_tool('get_grouped_schema')
class GetGroupedSchema(BaseTool):
    description = (
        'Returns the schema and usage instructions for the `grouped` SQL table. '
        'Use this when identifying and analyzing possibly duplicated invoices that have been clustered by similarity. '
        'Includes nested case and invoice data per group, as well as overpayment and confidence metrics.'
    )
    parameters = []

    def call(self, params: str, **kwargs) -> str:
        _ = json5.loads(params) if params else {}

        return json5.dumps({
            'schema': """
=== GROUPED TABLE ===

- group_id (VARCHAR): Unique identifier for each group (PK)
- amount_overpaid (BIGINT): Total overpaid amount for the group
- itemCount (BIGINT): Number of items in the group
- date (VARCHAR): Date of the group
- pattern (VARCHAR): Pattern used to group similar invoices. One of: 'Similar Value', 'Similar Reference', 'Exact Match', 'Similar Date', 'Similar Vendor', 'Multiple'
- open (BOOLEAN): Status of the group (open or closed)
- confidence (VARCHAR): Confidence level for the pattern ('High', 'Medium', 'Low')
- items (STRUCT[]): Array of grouped items, each containing:
    - id (INTEGER): Item ID (FK → invoices.id)
    - case (STRUCT): Contains case-level details:
        - id (VARCHAR): Case identifier
        - order_date (VARCHAR): Order date
        - employee_id (VARCHAR): Employee handling the case
        - branch (VARCHAR): Branch of the case
        - supplier (VARCHAR): Supplier name
        - avg_time (DOUBLE): Average duration of the case
        - estimated_delivery (VARCHAR): Estimated delivery
        - delivery (VARCHAR): Actual delivery
        - on_time (BOOLEAN): Whether delivered on time
        - in_full (BOOLEAN): Whether delivered in full
        - number_of_items (INTEGER): Total items
        - ft_items (INTEGER): Fast-track items
        - total_price (INTEGER): Case total price
    - date (VARCHAR): Item date
    - unit_price (VARCHAR): Unit price of the item
    - quantity (INTEGER): Quantity of the item
    - value (VARCHAR): Value of the item
    - pattern (VARCHAR): Pattern type ('Similar Value', etc.)
    - open (BOOLEAN): Item status
    - group_id (VARCHAR): Associated group ID
    - confidence (VARCHAR): Confidence in pattern match
    - description (VARCHAR): Description of item
    - payment_method (VARCHAR): Payment method used
    - pay_date (VARCHAR): Date of payment
    - special_instructions (VARCHAR): Additional notes
    - accuracy (INTEGER): Accuracy score of item-level pattern match
   
Instructions:
- Use table alias `g.` for `grouped`, `item.` for unnested item fields.
- Use `UNNEST(g.items) AS item` to access nested fields.
- Always apply TRIM() when comparing text values (e.g., pattern, supplier).
- Prefer flat `invoices` table for simpler queries.
- Filter early and use appropriate aliases and aggregation.

"""
        }, ensure_ascii=False)


@register_tool('get_invoices_schema')
class GetInvoicesSchema(BaseTool):
    description = (
        'Returns the schema and usage instructions for the `invoices` SQL table. '
        'Use this to query flat invoice records, including associated case metadata, pattern detection fields, and payment details.'
    )
    parameters = []

    def call(self, params: str, **kwargs) -> str:
        _ = json5.loads(params) if params else {}

        return json5.dumps({
            'schema': """
=== INVOICES TABLE ===


- id (BIGINT): Unique identifier for each invoice (PK)
- date (TIMESTAMP_NS): Invoice date and time
- unit_price (VARCHAR): Unit price of the item
- quantity (BIGINT): Number of items
- value (VARCHAR): Total invoice value
- pattern (VARCHAR): Pattern used to identify duplicates ('Similar Value', 'Similar Reference', 'Exact Match', 'Similar Date', 'Similar Vendor', 'Multiple')
- open (BOOLEAN): Status of the invoice (open or closed)
- group_id (VARCHAR): ID of the associated group (FK → grouped.group_id)
- confidence (VARCHAR): Confidence level for pattern classification ('High', 'Medium', 'Low')
- description (VARCHAR): Invoice description
- payment_method (VARCHAR): Payment method used
- pay_date (TIMESTAMP_NS): Date when payment was made
- special_instructions (VARCHAR): Additional notes
- accuracy (BIGINT): Accuracy score of invoice pattern match
- case_id (VARCHAR): Associated case ID
- case_order_date (TIMESTAMP_NS): Case order date
- case_employee_id (VARCHAR): Employee handling the case
- case_branch (VARCHAR): Branch handling the case
- case_supplier (VARCHAR): Supplier associated with the case
- case_avg_time (DOUBLE): Average case duration
- case_estimated_delivery (TIMESTAMP_NS): Estimated delivery for the case
- case_delivery (TIMESTAMP_NS): Actual delivery date
- case_on_time (BOOLEAN): Whether case was on time
- case_in_full (BOOLEAN): Whether case was delivered in full
- case_number_of_items (BIGINT): Number of items in the case
- case_ft_items (BIGINT): Number of full-time items in the case
- case_total_price (BIGINT): Total price of the case

Instructions:
- Use table alias `i.` for `invoices`.
- Use `TRIM()` when comparing string fields like supplier, pattern.
- Use this table when item- and group-level nesting is unnecessary.
"""
        }, ensure_ascii=False)

In [4]:

prompt_instruction = '''
After receiving the user's request, you should:
1. Identify the relevant SQL tables based on the user's query.
2. Retrieve the schema for those tables by calling the relevant schema-fetching tools (e.g., `get_cases_schema`, `get_activities_schema`, `get_variants_schema`).
3. Analyze the user's query and use the schema information to generate a prompt.
4. Provide a brief instruction about how to query the relevant tables based on the schema.
5. Return the table schemas where the query should be executed as well as its relevant columns with datatypes and descriptions. Do not Include any additional information.

The goal is to make the user request more specific by formulating a SQL query and instructions based on the relevant schemas of the tables.

Additional instructions:
- For invoices related questions try to avoid using the schema of the grouped table, if invoices table is enough for the query just use that one.
'''

tools = ['get_cases_schema', 'get_activities_schema', 'get_variants_schema','get_grouped_schema', 'get_invoices_schema', 'code_interpreter']  # Tools include schema fetchers and code interpreter
#files = ['./doc.pdf']  # You can provide a PDF file if necessary
prompt_agent = Assistant(llm=llm_cfg,
                system_message=prompt_instruction,
                function_list=tools,
                #files=files
                )

In [5]:
@register_tool('execute_sql_with_prompt')
class ExecuteSQLWithPrompt(BaseTool):
    description = 'Generates and executes a SQL query using a provided prompt and original user question. The prompt should describe what SQL to run.'

    parameters = [
        {
            'name': 'question',
            'type': 'string',
            'description': 'The original user question for context.',
            'required': True
        },
        {
            'name': 'prompt',
            'type': 'string',
            'description': 'The SQL prompt provided by another agent. It should describe what SQL to generate and run.',
            'required': True
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        question = args['question']
        prompt = args['prompt']

        # You can keep this if the SQL generator expects consistent formatting
        general_instructions = """/no_think
        You are an SQL assistant specialized in DuckDB. Your task is to generate accurate SQL queries based on natural language questions/tasks, following the schema and rules below.

        ### MAIN RULES:
        - Generate only one SQL Query.
        - The result must be executable as it is, so do not include any instructions, just the SQL code.
        - Only use the provided schemas to generate the SQL query, and do not reference any other tables or schemas.
        - You can perform JOINs between the tables, but you should not reference any other tables or schemas.
        - If the query is already given in this prompt you can use it as a basis and change it, for example to not select all columns but only the necessary ones.
        """
        start= time.time()
        combined_prompt = general_instructions + prompt

        # Assume this function executes the final query based on prompt and returns results
        result = run_sql_workflow(question, combined_prompt)
        end= time.time()
        print(f"\nExecution time for the workflow: {end - start} seconds")
        return result

In [6]:
sql_instruction = '''
You will receive a query and a prompt for SQL generation. Your need to::

1. Use the tool `execute_sql_with_prompt` which will generate and execute the SQL query based on the provided prompt and question.
2. Return the result of the SQL query execution as it is, without any additional instructions or comments.
'''

tools2 = ['execute_sql_with_prompt']  # Tools include schema fetchers and code interpreter

sql_bot = Assistant(llm=llm_cfg,
                system_message=sql_instruction,
                function_list=tools2,
                #files=files
                )


In [7]:
@register_tool('handoff_to_prompt_generator')
class HandoffToPromptAgent(BaseTool):
    description = 'Generates a prompt for the sub task that needs to be answered with a SQL query.'

    parameters = [
        {
            'name': 'task',
            'type': 'string',
            'description': 'The individual task that needs to be answered with a SQL query, no composed questions.',
            'required': True
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        start= time.time()
        args = json5.loads(params)
        task = args['task']        
        sup_message = {'role': 'user', 'content':task}
        # Assume this function executes the final query based on prompt and returns results
        for response in prompt_agent.run(messages=[sup_message]):
            response_plain_text = response[-1]["content"]
        response_plain_text = remove_think_blocks(response_plain_text)
        end= time.time()
        print(f"\nExecution time for prompt agent: {end - start} seconds")
        return response_plain_text

@register_tool('handoff_to_sql_generator')
class HandoffToSQLAgent(BaseTool):
    description = 'Generates and executes SQL queries for the given task based on the prompt.'

    parameters = [
        {
            'name': 'task',
            'type': 'string',
            'description': 'The individual task that needs to be answered with a SQL query, no composed questions, It needs to include the relevant context.',
            'required': True
        },
        {
            'name': 'prompt',
            'type': 'string',
            'description': 'The full exact prompt provided by a previous tool call `handoff_to_prompt_generator`. It should include the schemas of the tables and the instructions for generating queries.',
            'required': True
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        start= time.time()
        args = json5.loads(params)
        task = args['task']
        prompt= args['prompt']
        sup_message = {'role': 'user', 'content':task+prompt}
        # Assume this function executes the final query based on prompt and returns results
        for response in sql_bot.run(messages=[sup_message]):
            response_plain_text = response[-1]["content"]
        response_plain_text = remove_think_blocks(response_plain_text)
        end= time.time()
        print(f"\nExecution time for SQL agent: {end - start} seconds")
        print(response_plain_text)
        return response_plain_text
    
@register_tool("decompose_or_rewrite_question")
class DecomposeOrRewriteQuestion(BaseTool):
    description = (
        "Rewrites vague or complex user questions into clear, domain-specific, SQL-ready sub-questions "
        "based on process mining or supplier invoice deduplication tasks."
    )
    parameters = [
        {
            "name": "question",
            "type": "string",
            "description": "The original user query to interpret or decompose."
        },
        {
            "name": "chat_history",
            "type": "string",
            "description": "Recent chat history to help resolve references and context."
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        start= time.time()
        parsed = json5.loads(params)
        question = parsed["question"]
        chat_history = parsed.get("chat_history", "")

        llm= OllamaLLM(model="qwen3:4b",temperature=0.1)
        system_prompt = """
        /no_think
        You are a domain-aware assistant helping rephrase or decompose user questions into SQL-friendly subtasks for analytics.

        Use case is either: 
        - Process Mining (cases, activities, variants)
        - Supplier Invoice Deduplication (duplicate detection using pattern and confidence)

        Your job:
        - If the question is vague or indirect, rewrite it as a clear and specific analytical question.
        - If it involves multiple steps (comparisons, filters, deviations, KPIs), break it into simple measurable sub-questions.
        - Resolve vague time expressions (e.g., "recently" → "last 30 days") and references ("those", "they") using chat history.

        Output:
        - Return the sub-questions as a comma-separated list.
        - If the question is already clear and singular, return it as-is.

        Chat History:
        {chat_history}
        """
        reformat_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "User's question: {question}"),
            ]
        )
        reformatter = reformat_prompt | llm
        result = reformatter.invoke({"question": question, "chat_history": chat_history})
        result= remove_think_blocks(result)
        end= time.time()
        print(f"\nExecution time for Reformatter: {end - start} seconds")
        return result

@register_tool("generate_insights")
class GenerateInsights(BaseTool):
    description = "Generates actionable insights from the question, SQL result, and assistant answer."

    parameters = [
        {"name": "question", "type": "string", "description": "Original user question."},
        {"name": "answer", "type": "string", "description": "Final assistant answer."}
    ]

    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        question = args["question"]
        answer = args["answer"]

        system_prompt = """
        /no_think
        You are SOFIA, an experienced business consultant and analyst.
        
        You will be given:
        - A user’s business question
        - SQL result data (raw or summarized)
        - An assistant’s final answer based on the data
        
        Your job is to:
        1. Analyze the data through a business lens — identify trends, gaps, inefficiencies, or outliers.
        2. Derive a **concrete, high-value recommendation** the user can act on (e.g., improve vendor selection, reduce rework, increase automation, re-balance workflows).
        3. Keep the insight sharp, concise, and suitable for an executive audience.
        
        OUTPUT:
        Insight: <1-sentence meaningful suggestion focused on ROI, optimization, or risk mitigation>
        
        If data is inconclusive, say:
        No actionable insight found based on current data.
        """

        llm = OllamaLLM(model="qwen3:8b", temperature=0.3, top_p=0.95)
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "Please provide a 1-line actionable insight:")
        ])
        chain = prompt | llm | StrOutputParser()
        return chain.invoke({"question":question,"answer":answer}).strip()

In [8]:
supervisor_instruction='''
/no_think
You are the supervisor of the interaction between the user and two specialized agents.

Your goal is to answer the user's question, even if it requires multiple steps or SQL queries.

When you receive a user query:
1. Analyze whether it can be answered directly or if it needs to be broken down into multiple steps.
2. If multiple steps are required:
   - Break the query into clear subtasks.
   - For each subtask:
     a. Call the `handoff_to_prompt_generator` tool to generate a prompt.
     b. Then call the `handoff_to_sql_generator` tool generate and execute an SQL query based on the previous prompt.
3. If a single step is needed:
   - Do the same (generate prompt → generate and execute SQL).
4. Optionally analyze or summarize the results.
5. Combine the results from all subtasks and generate a final answer for the user.

Always return a final concise and insightful summary based on the results.

## IMPORTANT
- If multiple steps are required and some of those depend of the result of previous steps, you must execute those tool calls sequentially.
- Always make sure that for each step you generate it calls first the prompt agent and then with this prompt you call the execution agent.
- **DO NOT GUESS OR ASSUME THE RESULTS OF THE QUERIES, ALWAYS TRY TO EXECUTE THE QUERIES FIRST WITH THE CORRESPONDING TOOL**
- Avoid looping among tools

After answering the user's question, always call the tool `generate_insights` using:
- The original question
- The SQL result
- The assistant's answer

Append the generated insight to the final answer before returning it to the user.

'''

supervisor= Assistant(llm=llm_cfg,
                system_message=supervisor_instruction,
                function_list=['handoff_to_prompt_generator','handoff_to_sql_generator', 'generate_insights'],
                )

In [9]:
messages= []
query = 'What is the most frequent activity variant?'
# Append the user query to the chat history.
messages.append({'role': 'user', 'content': query})
response_plain_text = ''

for response in supervisor.run(messages=messages):
        response_plain_text = typewriter_print(response, response_plain_text)
response_plain_text = remove_think_blocks(response_plain_text)
messages.append({'role': 'assistant', 'content': response_plain_text})

<think>

</think>


[TOOL_CALL] handoff_to_prompt_generator
{"task": "Identify the most frequent activity variant."}
Execution time for prompt agent: 12.31419062614441 seconds

[TOOL_RESPONSE] handoff_to_prompt_generator
To identify the most frequent activity variant, we'll use the `variants` table and sort by the `number_cases` (or `percentage`) column to find the variant with the highest case count. Here's how to query it:

---

### **SQL Query**
```sql
SELECT id, activities, number_cases, percentage
FROM variants
ORDER BY number_cases DESC
LIMIT 1;
```

---

### **Relevant Columns in `variants` Table**
| Column Name      | Data Type   | Description                              |
|------------------|-------------|------------------------------------------|
| `id`             | BIGINT      | Unique identifier for the variant        |
| `activities`     | VARCHAR[]   | Ordered list of activities in the variant |
| `number_cases`   | BIGINT      | Number of cases following this variant 

2025-05-16 14:06:49,001 - agent.py - 188 - WARNING - An error occurred when calling tool `generate_insights`:
NameError: name 'StrOutputParser' is not defined
Traceback:
  File "/usr/local/lib/python3.10/dist-packages/qwen_agent/agent.py", line 178, in _call_tool
    tool_result = tool.call(tool_args, **kwargs)
  File "/tmp/ipykernel_7150/145928320.py", line 160, in call
    chain = prompt | llm | StrOutputParser()




[TOOL_RESPONSE] generate_insights
An error occurred when calling tool `generate_insights`:
NameError: name 'StrOutputParser' is not defined
Traceback:
  File "/usr/local/lib/python3.10/dist-packages/qwen_agent/agent.py", line 178, in _call_tool
    tool_result = tool.call(tool_args, **kwargs)
  File "/tmp/ipykernel_7150/145928320.py", line 160, in call
    chain = prompt | llm | StrOutputParser()

<think>

</think>

The most frequent activity variant has a **64.58%** share of total cases, representing the most common process path. Its activity sequence is:

**Order Creation → Order Approval → Send to Supplier → Get Confirmation from Supplier → Receive Shipment Confirmation from Supplier → Receive Invoice → Receive Materials → Verify Materials → Accept Materials → Integrate to Inventory → Payment → Distribute Materials**.

This variant dominates the process flow based on the provided data.

**Insight:**  
The most frequent activity variant represents the majority of the process flow, i